In [1]:
from collections import Counter
from matplotlib import pyplot as plt
from google.colab import drive
import os

#mount your Google drive into this notebook
drive.mount('/content/gdrive')
#find the path to your Google drive root
os.getcwd()+"/gdrive/My Drive"
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Data Mining/HW5')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
def readFile(file):
  with open(file, 'r') as f:
    data = f.readlines()
    return list(data[0])
stream1 = readFile('S1test.txt')  
stream2 = readFile('S2test.txt') 
print ('Stream 1:', len(stream2))
#print ('Stream 2:', stream2)
s1 = readFile('S1.txt')  
s2 = readFile('S2.txt') 

Stream 1: 69


In [20]:
#!pip install xxHash
import xxhash
class countMinSketch(object):
    stream = []
    m = 0
    k = 10
    t = 5
    Counters = []
    def __init__(self, path):
        with open(path, 'r') as file:
            for l in file: # Should only be one line, but just in case
                for char in l:
                    self.stream.append(char)
        self.m = len(self.stream)
    def initCounters(self):
        """
        Init  the counters to all 0's
        :return:
        """
        for i in range(self.t):
            self.Counters.append([])
            for j in range(self.k):
                self.Counters[i].append(0)
    def hashj(self, j, ai):
        """
        Returns a hash function of ai given j as the seed
        :param j: seed
        :param ai: item to hash
        :return: hash integer
        """
        h = xxhash.xxh32(seed=j)
        h.update(ai)
        hj = h.intdigest()%self.k
        return hj
    def countMinSketch(self):
        """
        Populates the counter table
        :return: nothing
        """
        self.initCounters()
        for i in range(self.m):
            for j in range(self.t):
                self.Counters[j][self.hashj(j, self.stream[i])] += 1
    def query(self, q):
        """
        Gets the frequency of the specified q
        :param q: the item to query
        :return: min frequency
        """
        m = self.Counters[0][self.hashj(0, q)]
        for j in range(self.t):
            current = self.Counters[j][self.hashj(j, q)]
            if m > current:
                m = current
        return m

#path1 = 'S1test.txt'
#path2 = 'S2test.txt'

path1 = 'S1.txt'
#path2 = 'S2.txt'

cms = countMinSketch(path1)
cms.countMinSketch()

print ('a - ' + str(cms.query('a')))

print ('b - ' + str(cms.query('b')))

print ('c - ' + str(cms.query('c')))

print ('The estimated ratio of a', " is " , round(cms.query('a')/cms.m,3) )
print ('The estimated ratio of b', " is " , round(cms.query('b')/cms.m,3) )
print ('The estimated ratio of c', " is " , round(cms.query('c')/cms.m,3) )

count = 0
for c in cms.stream:
    if c == 'a':
        count += 1
print ("actual count = " , count)
print(len(set(cms.stream)))
print (count+len(set(cms.stream))/10)
print ((count+len(set(cms.stream))/10)/cms.m)

a - 959642
b - 659576
c - 420301
The estimated ratio of a  is  0.32
The estimated ratio of b  is  0.22
The estimated ratio of c  is  0.14
actual count =  899566
22
899568.2
0.29985606666666664
